In [2]:
from binary_tree import TreeNode
from typing import Optional, List
import math
from math import inf
import collections
import numpy as np
from collections import defaultdict, Counter, deque
import itertools
import bisect
import heapq

## 二叉树

In [ ]:
class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        """
        计算二叉树的最大深度（递归）
        参数:
            root: 二叉树根节点
        返回:
            整数，树的最大深度（根节点深度为1）
        思路:
            递归地计算左右子树深度，取较大者加1；空节点深度为0
        """
        # 递归终止：如果节点为空，深度为0
        if not root:
            return 0
        # 递归计算左右子树深度并取最大值
        return max(self.maxDepth(root.left), self.maxDepth(root.right)) + 1

    def invertTree(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        """
        翻转二叉树（镜像）
        参数:
            root: 二叉树根节点
        返回:
            翻转后的根节点（就地修改）
        思路:
            递归翻转左右子树，然后交换左右子节点
        """
        if not root:
            return None
        # 递归翻转左子树和右子树
        left_inverted = self.invertTree(root.left)
        right_inverted = self.invertTree(root.right)
        # 交换左右子树
        root.left = right_inverted
        root.right = left_inverted
        return root

    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        """
        判断二叉树是否对称（镜像相同）
        参数:
            root: 二叉树根节点
        返回:
            布尔值，是否对称
        思路:
            定义递归函数比较左右子树：左右节点值相等且左.left vs 右.right，左.right vs 右.left 都对称
        """
        if not root:
            return True

        def recursive_check(
            left: Optional[TreeNode], right: Optional[TreeNode]
        ) -> bool:
            # 两个子树都为空 -> 对称
            if left is None and right is None:
                return True
            # 只有一个为空 -> 不对称
            if left is None or right is None:
                return False
            # 值不同 -> 不对称
            if left.val != right.val:
                return False
            # 递归比较外侧和内侧节点
            return recursive_check(left.left, right.right) and recursive_check(
                left.right, right.left
            )

        return recursive_check(root.left, root.right)

    diameter = 0
    def diameterOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        """
        二叉树的直径（节点数或边数的最长路径）
        参数:
            root: 二叉树根节点
        返回:
            树的直径长度（这里返回边数，所以最终减1）
        思路:
            使用后序遍历计算每个节点的左右子树深度，更新直径为左深度+右深度+1（节点数）
            最终返回节点数-1 得到边数
        注意:
            diameter 是类属性，会在多次调用间保留；若有多次调用建议先重置 self.diameter = 0
        """

        # 局部递归函数返回以 node 为根的最大深度（节点数）
        def postorder_traversal(node: Optional[TreeNode]) -> int:
            if not node:
                return 0
            l_depth = postorder_traversal(node.left)
            r_depth = postorder_traversal(node.right)
            # 当前节点作为路径中间点时的路径节点数
            self.diameter = max(self.diameter, l_depth + r_depth + 1)
            # 向上返回节点数（深度）
            return max(l_depth, r_depth) + 1

        postorder_traversal(root)
        # 转换为边数：节点数 - 1；若树为空 self.diameter 初始为0，返回 -1 表示无边
        return self.diameter - 1

    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        """
        二叉树的层序遍历（每层节点值的列表）
        参数:
            root: 二叉树根节点
        返回:
            嵌套列表：每个子列表为一层的节点值（从上到下，从左到右）
        思路:
            使用队列进行 BFS，按层处理节点
        """
        if not root:
            return []
        result = []
        queue = deque([root])
        while queue:
            level_size = len(queue)
            level_res = []
            for _ in range(level_size):
                node = queue.popleft()
                level_res.append(node.val)
                # 将下一层节点入队
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
            result.append(level_res)
        return result

    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        """
        将有序数组转换为高度平衡的二叉搜索树（BST）
        参数:
            nums: 已升序排列的整数数组
        返回:
            构造的 BST 根节点
        思路:
            选择中间元素作为根，递归构造左右子树，保证左右高度差不大于1
        复杂度:
            时间 O(n)，空间 O(log n)（递归栈）
        """
        if not nums:
            return None
        mid_idx = len(nums) // 2
        mid_val = nums[mid_idx]
        root = TreeNode(val=mid_val)
        # 左半部分构成左子树，右半部分构成右子树
        root.left = self.sortedArrayToBST(nums[:mid_idx])
        root.right = self.sortedArrayToBST(nums[mid_idx + 1 :])
        return root

    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        """
        判断一棵树是否为二叉搜索树（BST）
        参数:
            root: 二叉树根节点
        返回:
            布尔值，是否为 BST
        思路:
            使用递归并维护当前节点允许的取值区间 (low, high)
            每个节点必须满足 low < node.val < high；递归更新区间
        """

        def helper(node: Optional[TreeNode], low: float, high: float) -> bool:
            if not node:
                return True
            if not (low < node.val < high):
                return False
            return helper(node.left, low, node.val) and helper(
                node.right, node.val, high
            )

        return helper(root, -math.inf, math.inf)

    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        """
        二叉搜索树中第 k 小的元素（中序遍历升序）
        参数:
            root: BST 根节点
            k: 第 k 小（1-based）
        返回:
            第 k 小的值；若不存在则返回 None
        思路:
            中序遍历并计数，找到第 k 个节点即返回
        """
        self.count = 0
        self.res = None

        def inorder_traversal(node: Optional[TreeNode]):
            if not node or self.res is not None:
                return
            inorder_traversal(node.left)
            self.count += 1
            if self.count == k:
                self.res = node.val
                return
            inorder_traversal(node.right)

        inorder_traversal(root)
        return self.res

    def rightSideView(self, root: Optional[TreeNode]) -> List[int]:
        """
        二叉树的右视图（从右侧能看到的每一层最右节点）
        参数:
            root: 二叉树根节点
        返回:
            列表，每层从右侧看到的节点值（从上到下）
        思路:
            层序遍历，每层记录最后一个出队节点的值
        """
        if not root:
            return []
        result = []
        queue = deque([root])
        while queue:
            level_size = len(queue)
            for idx in range(level_size):
                node = queue.popleft()
                # 每层最后一个节点即为从右侧看到的节点
                if idx == level_size - 1:
                    result.append(node.val)
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
        return result

    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        """
        从前序和中序遍历序列重建二叉树
        参数:
            preorder: 前序遍历列表（根-左-右）
            inorder: 中序遍历列表（左-根-右）
        返回:
            重建后的树根节点；若使用特殊约定(-1)则返回 TreeNode(-1)
        思路:
            前序第一个元素为根，在中序中定位根，将中序分为左/右子序列，
            前序对应切片用于构造左右子树，递归完成重建
        """
        # 特殊约定处理（保留原逻辑）
        if not preorder or not inorder:
            return None
        if preorder[0] == -1 or inorder[0] == -1:
            return TreeNode(-1)

        def build_tree(pre_l: List[int], in_l: List[int]) -> Optional[TreeNode]:
            if not pre_l or not in_l:
                return None
            root_val = pre_l[0]
            root = TreeNode(root_val)
            # 在中序中找到根的位置以划分左右子树
            root_idx = in_l.index(root_val)
            left_in = in_l[:root_idx]
            right_in = in_l[root_idx + 1 :]
            left_pre = pre_l[1 : 1 + len(left_in)]
            right_pre = pre_l[1 + len(left_in) :]
            root.left = build_tree(left_pre, left_in)
            root.right = build_tree(right_pre, right_in)
            return root

        return build_tree(preorder, inorder)

    def pathSum(self, root: Optional[TreeNode], targetSum: int) -> int:
        """
        计算二叉树中路径和等于 targetSum 的路径数量（路径必须向下）
        参数:
            root: 二叉树根节点
            targetSum: 目标路径和
        返回:
            满足条件的路径数量（整型）
        思路:
            使用前缀和 + 哈希表记录前缀和出现次数，深度优先遍历：
            当前前缀和 curr_sum，若存在 prefix 使 curr_sum - prefix == targetSum，则找到路径
            回溯时撤销当前前缀和的计数
        复杂度:
            时间 O(n)，空间 O(n)
        """

        def dfs(node: Optional[TreeNode], curr_sum: int, prefix_sums: dict) -> int:
            if not node:
                return 0
            curr_sum += node.val
            # 统计以当前节点为终点的、和为 targetSum 的路径数量
            count = prefix_sums.get(curr_sum - targetSum, 0)
            # 将当前前缀和加入哈希表
            prefix_sums[curr_sum] = prefix_sums.get(curr_sum, 0) + 1
            # 递归左右子树累加结果
            count += dfs(node.left, curr_sum, prefix_sums)
            count += dfs(node.right, curr_sum, prefix_sums)
            # 回溯：撤销当前前缀和计数
            prefix_sums[curr_sum] -= 1
            return count

        # 初始前缀和0出现1次
        return dfs(root, 0, {0: 1})

    max_path_sum = -inf

    def maxPathSum(self, root: Optional[TreeNode]) -> int:
        """
        二叉树的最大路径和（路径可以从任意节点出发并终止于任意节点，中间不必经过根）
        参数:
            root: 二叉树根节点
        返回:
            最大路径和（整数）
        思路:
            后序遍历，每个节点计算左右子树向上贡献的最大值（若为负则取0），
            更新全局最大路径为 node.val + left_gain + right_gain；
            向上返回 node.val + max(left_gain, right_gain)（路径不能分叉）
        注意:
            max_path_sum 为类属性，若多次调用请先重置 self.max_path_sum = -inf
        """

        def dfs(node: Optional[TreeNode]) -> int:
            if not node:
                return 0
            left_max = dfs(node.left)
            right_max = dfs(node.right)
            # 向上可贡献的最大值，若为负则不贡献（取0）
            left_gain = max(0, left_max)
            right_gain = max(0, right_max)
            # 当前节点能够组成的最大路径和（可能经过左右子树）
            curr_max_path = node.val + left_gain + right_gain
            # 更新全局最大路径和
            self.max_path_sum = max(self.max_path_sum, curr_max_path)
            # 向父节点返回的最大贡献（路径不能分叉）
            return node.val + max(left_gain, right_gain)

        dfs(root)
        return self.max_path_sum

## 图论

In [ ]:
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        """
        计算一个由 '1'（陆地）和 '0'（水）组成的二维网格中岛屿的数量。
        岛屿是由水平或垂直相邻的 '1' 连接而成的。

        Args:
            grid: 一个二维列表，代表地图网格。

        Returns:
            网格中岛屿的数量。
        """
        # 初始化岛屿数量为 0
        islands = 0

        # 检查网格是否为空，如果为空则直接返回 0
        if not grid or not grid[0]:
            return islands

        def dfs_recursive(grid, row, col):
            """
            使用深度优先搜索（DFS）来遍历并淹没一个岛屿。
            这个函数会递归地将所有与 (row, col) 相连的陆地 ('1') 变成水 ('0')。

            Args:
                grid: 地图网格。
                row: 当前单元格的行索引。
                col: 当前单元格的列索引。
            """
            # 递归的终止条件：
            # 1. 行索引越界（小于0或大于等于总行数）
            # 2. 列索引越界（小于0或大于等于总列数）
            # 3. 当前单元格不是陆地 ('1')
            if (
                row < 0
                or row >= len(grid)
                or col < 0
                or col >= len(grid[0])
                or grid[row][col] != "1"
            ):
                return

            # 将当前陆地单元格标记为 '0'，表示已经访问过（淹没）
            grid[row][col] = "0"

            # 递归地访问当前单元格的四个方向（上、下、左、右）
            dfs_recursive(grid, row - 1, col)  # 上
            dfs_recursive(grid, row + 1, col)  # 下
            dfs_recursive(grid, row, col - 1)  # 左
            dfs_recursive(grid, row, col + 1)  # 右

        # 获取网格的行数和列数
        rows, cols = len(grid), len(grid[0])
        # 遍历网格中的每一个单元格
        for i in range(rows):
            for j in range(cols):
                # 如果发现一个单元格是陆地 ('1')
                if grid[i][j] == "1":
                    # 从这个单元格开始进行深度优先搜索，淹没整个岛屿
                    dfs_recursive(grid, i, j)
                    # 淹没一个完整的岛屿后，岛屿数量加一
                    islands += 1

        # 返回最终统计的岛屿数量
        return islands

    def orangesRotting(self, grid: List[List[int]]) -> int:
        """
        计算所有新鲜橘子腐烂所需的最短时间。
        网格中 0 代表空格，1 代表新鲜橘子，2 代表腐烂的橘子。
        每分钟，任何与腐烂橘子相邻（上、下、左、右）的新鲜橘子都会腐烂。
        此问题适合使用广度优先搜索（BFS），因为我们需要找到最短时间。

        Args:
            grid: 一个包含 0, 1, 2 的二维列表。

        Returns:
            所有新鲜橘子变腐烂所需的最小分钟数。如果不可能全部腐烂，则返回 -1。
        """
        # fresh: 新鲜橘子的数量
        fresh = 0
        # min_time: 已经过的时间（分钟）
        min_time = 0
        # queue: 用于广度优先搜索的双端队列
        queue = deque()

        rows, cols = len(grid), len(grid[0])
        # directions: 定义四个搜索方向（上、下、左、右）
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        # 如果网格为空，直接返回 0
        if not grid or not grid[0]:
            return min_time

        # 第一次遍历网格：
        # 1. 找到所有初始的腐烂橘子，将它们的坐标加入队列
        # 2. 统计所有新鲜橘子的数量
        for row in range(rows):
            for col in range(cols):
                if grid[row][col] == 2:
                    queue.append((row, col))
                if grid[row][col] == 1:
                    fresh += 1

        # 如果一开始就没有新鲜橘子，则不需要时间，返回 0
        if not fresh:
            return 0

        # 开始广度优先搜索
        # 当队列不为空时，表示还有腐烂的橘子可以影响周围
        while queue:
            # 当前队列的大小代表了在这一分钟内，所有会扩散的腐烂橘子
            level_size = len(queue)

            # 如果还有新鲜橘子，并且队列非空（意味着将要发生腐烂），时间就加一分钟
            # 这个判断确保了在最后一轮腐烂结束后，时间不会多加。
            if fresh > 0:
                min_time += 1

            # 遍历当前层级（即这一分钟内）的所有腐烂橘子
            for _ in range(level_size):
                # 从队列中取出一个腐烂橘子的坐标
                r, c = queue.popleft()
                # 检查它的四个方向
                for dr, dc in directions:
                    row, col = r + dr, c + dc

                    # 检查新坐标是否在网格内，并且该位置是一个新鲜橘子
                    if 0 <= row < rows and 0 <= col < cols and grid[row][col] == 1:
                        # 将新鲜橘子变为腐烂橘子
                        grid[row][col] = 2
                        # 将这个新腐烂的橘子加入队列，以便在下一分钟扩散
                        queue.append((row, col))
                        # 新鲜橘子的数量减一
                        fresh -= 1

        # BFS 结束后，检查是否还有新鲜橘子
        # 如果 fresh == 0，说明所有橘子都已腐烂，返回总时间
        if fresh == 0:
            return min_time
        # 否则，说明有无法被腐烂的橘子，返回 -1
        else:
            return -1

    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        """
        判断给定的课程和它们的先修课程关系是否可能完成所有课程。
        这本质上是一个检测有向图中是否存在环的问题。如果存在环，则无法完成所有课程。

        Args:
            numCourses: 课程总数。
            prerequisites: 一个列表，每个元素 [a, b] 表示课程 a 的先修课程是 b。

        Returns:
            如果可以完成所有课程，返回 True；否则返回 False。
        """
        # 构建邻接表来表示课程依赖关系
        # graph[i] 存储了课程 i 的所有先修课程
        graph = [[] for _ in range(numCourses)]
        for course, pre in prerequisites:
            graph[course].append(pre)

        # visited 数组用于记录每个节点（课程）的访问状态
        # 0: 未访问 (unvisited)
        # 1: 正在访问 (visiting)，表示该节点在当前的 DFS 递归栈中
        # 2: 已完成访问 (visited)，表示该节点及其所有后续节点都已访问完毕且无环
        visited = [0] * numCourses

        def dfs(course):
            """
            通过深度优先搜索检测从当前课程出发是否存在环。

            Args:
                course: 当前正在访问的课程。

            Returns:
                如果从该课程出发的路径无环，返回 True；否则返回 False。
            """
            # 如果在本次 DFS 中再次遇到一个“正在访问”的节点，
            # 说明找到了一个环 (e.g., A -> B -> C -> A)，直接返回 False。
            if visited[course] == 1:
                return False

            # 如果该节点已经完成了访问（状态为2），说明从它出发的路径是安全的（无环），
            # 无需重复搜索，直接返回 True。
            if visited[course] == 2:
                return True

            # 将当前课程标记为“正在访问”
            visited[course] = 1

            # 递归地访问当前课程的所有先修课程
            for pre in graph[course]:
                if not dfs(pre):  # 如果在任何一个先修课程的路径中发现了环
                    return False  # 则立即返回 False

            # 如果当前课程的所有先修课程及其路径都已成功访问且无环，
            # 那么将当前课程标记为“已完成访问”，并返回 True。
            visited[course] = 2
            return True

        # 遍历每一门课程，以其为起点进行 DFS 检测
        for i in range(numCourses):
            if not dfs(i):  # 只要发现任何一个起点导致了环的存在
                return False  # 就说明课程计划是不可行的

        # 如果所有课程都检测完毕且没有发现环，则返回 True
        return True


class Trie:
    """
    前缀树 (Trie) 的实现。
    前缀树是一种用于高效存储和检索字符串集合的数据结构。
    """

    def __init__(self):
        """
        初始化前缀树，创建一个空的根节点。
        """
        self.root = TreeNode()

    def insert(self, word: str) -> None:
        """
        向前缀树中插入一个单词。

        Args:
            word: 要插入的字符串。
        """
        # 从根节点开始
        curr = self.root
        # 遍历单词中的每一个字符
        for c in word:
            # 如果当前节点的子节点中不存在该字符
            if c not in curr.son:
                # 创建一个新的节点作为子节点
                curr.son[c] = TreeNode()
            # 移动到子节点
            curr = curr.son[c]
        # 遍历结束后，将最后一个字符对应的节点标记为单词结尾
        curr.end = True

    def find(self, word: str) -> int:
        """
        在前缀树中查找一个字符串（可以是完整单词或前缀）。

        Args:
            word: 要查找的字符串。

        Returns:
            0: 如果字符串既不是单词也不是任何单词的前缀。
            1: 如果字符串是一个前缀，但不是一个完整的单词。
            2: 如果字符串是一个完整的单词。
        """
        # 从根节点开始
        curr = self.root
        # 遍历字符串中的每一个字符
        for c in word:
            # 如果在路径中找不到某个字符，说明该字符串不存在于树中
            if c not in curr.son:
                return 0  # 既不是单词也不是前缀
            # 移动到下一个节点
            curr = curr.son[c]

        # 遍历完成后，curr 指向字符串最后一个字符对应的节点
        # 如果该节点的 end 标记为 True，说明它是一个完整的单词
        # 否则，它只是一个前缀
        return 2 if curr.end else 1

    def search(self, word: str) -> bool:
        """
        搜索前缀树中是否存在一个完整的单词。

        Args:
            word: 要搜索的单词。

        Returns:
            如果单词存在，返回 True；否则返回 False。
        """
        # 调用 find 方法，如果返回值为 2，则说明找到了一个完整的单词
        return self.find(word) == 2

    def startsWith(self, prefix: str) -> bool:
        """
        检查前缀树中是否存在以给定前缀开头的单词。

        Args:
            prefix: 要检查的前缀。

        Returns:
            如果存在以该前缀开头的单词，返回 True；否则返回 False。
        """
        # 调用 find 方法，只要返回值不为 0（即为1或2），
        # 就说明这个字符串是树中某个单词的前缀（或者本身就是一个单词）
        return self.find(prefix) != 0

## 双指针

In [ ]:
class Solution:
    def trap(self, height):
        if not height or len(height) < 3:
            return 0

        # 初始化左右指针
        left = 0  # 左指针，从数组开始
        right = len(height) - 1  # 右指针，从数组末尾
        left_max = 0  # 记录左侧最大高度
        right_max = 0  # 记录右侧最大高度
        water = 0  # 累计接雨水量

        # 当左右指针未相遇时继续
        while left < right:
            # 比较左右指针指向的高度
            if height[left] < height[right]:
                # 左侧高度较小，处理左侧
                if height[left] >= left_max:
                    # 更新左侧最大高度
                    left_max = height[left]
                else:
                    # 当前位置可以接雨水
                    # 雨水量 = 左侧最大高度 - 当前高度
                    water += left_max - height[left]
                left += 1  # 左指针右移
            else:
                # 右侧高度较小或相等，处理右侧
                if height[right] >= right_max:
                    # 更新右侧最大高度
                    right_max = height[right]
                else:
                    # 当前位置可以接雨水
                    # 雨水量 = 右侧最大高度 - 当前高度
                    water += right_max - height[right]
                right -= 1  # 右指针左移

        return water

## 子串

In [ ]:
class Solution:
    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        # 使用双端队列存储数组索引，保持队列中索引对应的值单调递减
        # 保证队列中队首索引对应的值始终是最大值
        dq = deque()
        result = []

        # 窗口边界：[i-k+1 : i]
        for i in range(len(nums)):
            # 移除队列中超出窗口范围的索引
            while dq and dq[0] < (i - k + 1):
                dq.popleft()

            # 维护队列单调性，移除所有小于当前元素的索引
            while dq and nums[dq[-1]] < nums[i]:
                dq.pop()

            # 将当前索引加入队列
            dq.append(i)

            # 当窗口大小达到k时，开始记录最大值
            if i >= k - 1:
                # 队列头部索引对应的值就是当前窗口的最大值
                result.append(nums[dq[0]])

        return result

    def minWindow(self, s: str, t: str) -> str:
        if not s or not t or len(s) < len(t):
            return ""

        target_counts = collections.Counter(t)
        window = defaultdict(int)
        left = right = 0
        matched_count = 0

        min_len = inf
        start_idx = 0

        while right < len(s):
            char_r = s[right]
            right += 1

            # 更新窗口数据
            if char_r in target_counts:
                window[char_r] += 1
                # 如果当前字符的计数首次达到需求数量
                if window[char_r] == target_counts[char_r]:
                    matched_count += 1

            # left指针收缩：窗口已经覆盖所有t中字符
            while matched_count == len(target_counts):
                # 记录当前窗口长度，与min len比较并更新
                curr_len = right - left
                if curr_len < min_len:
                    min_len = curr_len
                    start_idx = left

                char_l = s[left]
                left += 1

                if char_l in target_counts:
                    # 注意：这里是先判断是否减少了 match_count，再减少 window[char_l]
                    # 因为如果 window[char_l] 减少后小于 needs[char_l]，则 match_count 也要相应减少
                    if (
                        window[char_l] == target_counts[char_l]
                    ):  # 如果当前字符的计数即将低于需求数量
                        matched_count -= 1
                    window[char_l] -= 1

        if min_len == -1:
            return ""
        else:
            return s[start_idx : start_idx + min_len]

## 普通数组

In [ ]:
class Solution:
    def firstMissingPositive(self, nums: List[int]) -> int:
        n = len(nums)

        # 非正数和大于n的数替换
        for i in range(n):
            if nums[i] <= 0 or nums[i] > n:
                nums[i] = inf

        # 使用数组索引作为哈希表，标记数字的存在性
        # 都标记为 -num , 作为存在的证明
        for i in range(n):
            num = abs(nums[i])
            if num <= n:
                nums[num - 1] = -abs(nums[num - 1])

        # 找到第一个正数位置，即为缺失的第一个正数
        for i in range(n):
            if nums[i] > 0:
                return i + 1

        return n + 1

## 回溯

In [ ]:
class Solution:
    """
    回溯法是一种通过探索所有可能的候选解来找出所有解的算法。
    如果候选解被确认不是一个解（或者至少不是最后一个解），回溯算法会通过在上一步进行一些变化来丢弃该解，即回溯。
    """
    def permute(self, nums: List[int]) -> List[List[int]]:
        """
        生成给定整数数组的全排列。

        Args:
            nums: 一个不含重复数字的整数数组。

        Returns:
            一个包含所有可能排列的列表。
        """
        result = []  # 存储所有找到的全排列
        path = []    # 存储当前正在构建的排列
        # `used` 数组用于标记 `nums` 中的元素是否已被使用，防止重复选择
        used = [False] * len(nums)

        def backtrack():
            """
            回溯核心函数，用于探索所有可能的排列。
            """
            # 递归终止条件：当当前路径的长度等于原始数组的长度时，
            # 说明找到了一个完整的排列。
            if len(path) == len(nums):
                # 将当前路径的一个副本添加到结果列表中。
                # 必须添加副本 (path[:])，因为 `path` 列表在后续的回溯过程中会被修改。
                result.append(path[:])
                return

            # 遍历 `nums` 中的所有数字
            for i in range(len(nums)):
                # 如果当前数字已经被使用过，则跳过
                if used[i]:
                    continue

                # --- 做出选择 ---
                # 将当前数字添加到路径中
                path.append(nums[i])
                # 标记该数字为已使用
                used[i] = True

                # --- 进入下一层决策树 ---
                # 递归调用，继续构建排列的下一个位置
                backtrack()

                # --- 撤销选择 (回溯) ---
                # 将刚刚添加的数字从路径中移除
                path.pop()
                # 将该数字的“已使用”状态重置为 False，以便在其他分支中可以再次使用
                used[i] = False

        # 启动回溯过程
        backtrack()
        return result

    def subsets(self, nums: List[int]) -> List[List[int]]:
        """
        生成给定整数数组的所有子集（幂集）。

        Args:
            nums: 一个整数数组。

        Returns:
            一个包含所有可能子集的列表。
        """
        result = []  # 存储所有找到的子集
        sub = []     # 存储当前正在构建的子集

        def backtrack(start_index):
            """
            回溯核心函数，用于探索所有可能的子集。
            
            Args:
                start_index: 本轮选择的起始索引，确保组合中的元素不重复。
            """
            # 每个节点（无论是否叶子节点）都是一个合法的子集，所以先将当前子集加入结果
            result.append(sub[:])

            # 从 `start_index` 开始遍历，为当前子集选择新的元素
            for i in range(start_index, len(nums)):
                # --- 做出选择 ---
                sub.append(nums[i])
                # --- 进入下一层决策树 ---
                # 递归调用，起始索引为 `i + 1`，因为每个数字在子集中只能使用一次
                backtrack(i + 1)
                # --- 撤销选择 (回溯) ---
                sub.pop()

        # 从索引 0 开始启动回溯过程
        backtrack(0)
        return result

    def letterCombinations(self, digits: str) -> List[str]:
        """
        给定一个仅包含数字 2-9 的字符串，返回所有它能表示的字母组合。

        Args:
            digits: 数字字符串。

        Returns:
            一个包含所有可能字母组合的列表。
        """
        if not digits:
            return []
            
        DIG_MAP = {
            "2": "abc", "3": "def", "4": "ghi", "5": "jkl",
            "6": "mno", "7": "pqrs", "8": "tuv", "9": "wxyz",
        }
        result = [] # 存储最终结果
        path = []   # 存储当前构建的组合

        def backtrack(index):
            """
            回溯核心函数。
            
            Args:
                index: 当前正在处理的 `digits` 字符串的索引。
            """
            # 递归终止条件：当前组合的长度等于数字字符串的长度
            if len(path) == len(digits):
                result.append("".join(path))
                return

            # 获取当前数字对应的字母集
            current_digit = digits[index]
            letters = DIG_MAP[current_digit]
            
            # 遍历字母集，进行选择
            for char in letters:
                path.append(char)          # 做出选择
                backtrack(index + 1)       # 递归处理下一个数字
                path.pop()                 # 撤销选择

        backtrack(0) # 从第一个数字开始
        return result

    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        """
        找出 `candidates` 中所有可以使数字和为 `target` 的组合。
        `candidates` 中的同一个数字可以无限制重复被选取。

        Args:
            candidates: 无重复元素的整数数组。
            target: 目标和。

        Returns:
            一个包含所有有效组合的列表。
        """
        result = [] # 存储最终结果
        path = []   # 存储当前组合

        def backtrack(start, curr_sum):
            """
            回溯核心函数。
            
            Args:
                start: 本轮选择的起始索引，用于去重。
                curr_sum: 当前路径中元素的和。
            """
            # 递归终止条件 1：当前和等于目标值，找到一个有效组合
            if curr_sum == target:
                result.append(path[:])
                return
            # 剪枝操作：如果当前和已经大于目标值，后续无需再添加元素
            elif curr_sum > target:
                return

            # 从 `start` 索引开始遍历候选数，避免产生重复组合
            for i in range(start, len(candidates)):
                path.append(candidates[i]) # 做出选择
                # 递归调用，起始索引仍为 `i`，因为数字可以重复使用
                backtrack(i, curr_sum + candidates[i])
                path.pop()                 # 撤销选择

        backtrack(0, 0) # 从索引0，和为0开始
        return result

    def generateParenthesis(self, n: int) -> List[str]:
        """
        生成 n 对括号的所有有效的（格式正确的）组合。

        Args:
            n: 括号的对数。

        Returns:
            一个包含所有有效括号组合的列表。
        """
        result = []

        def backtrack(curr, left_count, right_count):
            """
            回溯核心函数。
            
            Args:
                curr: 当前构建的括号字符串。
                left_count: 已使用的左括号数量。
                right_count: 已使用的右括号数量。
            """
            # 递归终止条件：字符串长度达到 2*n
            if len(curr) == 2 * n:
                result.append(curr)
                return

            # 剪枝/约束条件 1：如果左括号数量小于 n，可以添加一个左括号
            if left_count < n:
                backtrack(curr + "(", left_count + 1, right_count)

            # 剪枝/约束条件 2：如果右括号数量小于左括号数量，可以添加一个右括号
            # 这保证了括号的有效性
            if right_count < left_count:
                backtrack(curr + ")", left_count, right_count + 1)

        backtrack("", 0, 0)
        return result

    def exist(self, board: List[List[str]], word: str) -> bool:
        """
        在二维网格中查找一个单词，单词路径可以从任意单元格开始，
        通过上、下、左、右移动，且同一单元格不能重复使用。

        Args:
            board: 字符二维网格。
            word: 要查找的单词。

        Returns:
            如果找到单词，返回 True，否则返回 False。
        """
        m, n = len(board), len(board[0])

        def dfs_recursive(i, j, k):
            """
            深度优先搜索函数。
            
            Args:
                i, j: 当前在网格中的坐标。
                k: 当前正在匹配 `word` 中的第 k 个字符。
            """
            # 递归终止条件 1：越界或当前字符不匹配
            if i < 0 or i >= m or j < 0 or j >= n or board[i][j] != word[k]:
                return False

            # 递归终止条件 2：已成功匹配 `word` 的最后一个字符
            if k == len(word) - 1:
                return True

            # --- 做出选择 ---
            # 临时修改当前单元格的值，标记为已访问，防止重复使用
            temp = board[i][j]
            board[i][j] = "#"

            # --- 进入下一层决策树 ---
            # 递归地向四个方向搜索下一个字符
            found = (
                dfs_recursive(i + 1, j, k + 1)
                or dfs_recursive(i - 1, j, k + 1)
                or dfs_recursive(i, j + 1, k + 1)
                or dfs_recursive(i, j - 1, k + 1)
            )

            # --- 撤销选择 (回溯) ---
            # 恢复当前单元格的值，以便其他搜索路径可以使用它
            board[i][j] = temp

            return found

        # 遍历网格中的每一个单元格，作为搜索的起点
        for i in range(m):
            for j in range(n):
                # 如果从 (i, j) 出发能找到单词，立即返回 True
                if dfs_recursive(i, j, 0):
                    return True

        # 如果遍历完所有起点都找不到，返回 False
        return False
    
    def partition(self, s: str) -> List[List[str]]:
        """
        将一个字符串分割成若干个子串，使得每个子串都是回文串。
        返回所有可能的回文分割方案。

        Args:
            s: 输入字符串。

        Returns:
            一个包含所有回文分割方案的列表。
        """
        result = []
        sub = [] # 存储当前分割方案
        
        def is_palindrome(string):
            """辅助函数，检查一个字符串是否是回文串。"""
            return string == string[::-1]
        
        def backtrack(index):
            """
            回溯核心函数。
            
            Args:
                index: 当前分割的起始位置。
            """
            # 递归终止条件：如果起始位置到达字符串末尾，说明找到了一种完整的分割方案
            if index == len(s):
                result.append(sub[:])
                return
            
            # 尝试从 `index` 到字符串末尾的所有可能的分割点 `i`
            for i in range(index, len(s)):
                # 获取从 `index` 到 `i` 的子串
                substring = s[index : i + 1]
                # 如果这个子串是回文串
                if is_palindrome(substring):
                    sub.append(substring)    # 做出选择：将此回文子串加入当前方案
                    backtrack(i + 1)         # 递归地从 `i+1` 开始寻找下一个回文子串
                    sub.pop()                # 撤销选择：回溯

        backtrack(0)
        return result

    def solveNQueens(self, n: int) -> List[List[str]]:
        # 存储所有可行的解决方案
        result = []
        # 记录每行皇后所在的列位置，col[r]表示第r行的皇后在第col[r]列
        col = [0] * n

        # 深度优先搜索函数
        # r: 当前正在处理的行
        # rest_cols: 剩余可选的列集合，用集合数据结构便于快速删除和查找
        def dfs(r, rest_cols):
            # 递归终止条件：当处理完所有行（r == n）时，找到一个有效解
            if r == n:
                # 将解转换为题目要求的字符串格式
                # 对于每一行，生成一个长度为n的字符串：
                # '.'*c 表示皇后前面的空位
                # 'Q' 表示皇后位置
                # '.'*(n-c-1) 表示皇后后面的空位
                result.append(
                    ['.'*c + 'Q' + '.'*(n-c-1) for c in col]
                )
                return

            # 遍历当前行所有可选的列位置
            for c in rest_cols:
                # 检查当前位置是否合法（不与已放置的皇后冲突）
                # 对于每个已放置的皇后(R, col[R])，检查是否与当前皇后(r, c)冲突：
                # r+c != R+col[R] 检查是否在同一个主对角线（行索引与列索引之和相等）
                # r-c != R-col[R] 检查是否在同一个副对角线（行索引与列索引之差相等）
                # all(...) 确保与之前所有行的皇后都不冲突
                if all(r+c != R+col[R] and r-c!=R-col[R] for R in range(r)):
                    # 位置合法，将第r行的皇后放在第c列
                    col[r] = c
                    # 递归处理下一行，同时从可选列集合中移除已使用的列c
                    dfs(r+1, rest_cols-{c})

        # 从第0行开始搜索，初始可选列集合为所有列{0,1,2,...,n-1}
        dfs(0, set(range(n)))
        return result

## 二分查找

In [ ]:
class Solution:
    """
    一个包含多种基于二分查找及其变体算法解法的类。
    """

    # 版本一：使用 Python 内置的 bisect 模块
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        """
        在排序数组中查找给定目标值的第一个和最后一个位置。
        这个实现利用了 Python 的 `bisect` 模块，代码非常简洁。

        Args:
            nums: 一个已排序的整数数组。
            target: 要查找的目标值。

        Returns:
            一个包含起始和结束索引的列表 [start, end]，如果未找到则返回 [-1, -1]。
        """
        # `bisect.bisect_left` 查找 target 在 nums 中可以插入的第一个位置，
        # 也就是 target 的起始位置（如果存在）。
        start = bisect.bisect_left(nums, target)

        # 检查 target 是否真的存在于数组中。
        # 条件1: `start == len(nums)` 表示 target 大于所有元素，插入点在末尾。
        # 条件2: `nums[start] != target` 表示 target 不在数组中（插入点在两个数之间）。
        # 如果任一条件为真，说明 target 不存在。
        if start == len(nums) or nums[start] != target:
            return [-1, -1]
        
        # `bisect.bisect_right` 查找 target 在 nums 中可以插入的最右侧位置。
        # 这个位置的索引减 1，就是 target 的最后一个出现位置。
        end = bisect.bisect_right(nums, target) - 1
        
        return [start, end]

    # 版本二：手动实现二分查找
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        """
        在排序数组中查找给定目标值的第一个和最后一个位置。
        这个实现通过两次修改版的二分查找来分别确定左边界和右边界。

        Args:
            nums: 一个已排序的整数数组。
            target: 要查找的目标值。

        Returns:
            一个包含起始和结束索引的列表 [start, end]，如果未找到则返回 [-1, -1]。
        """
        def find_first_position(nums, target):
            """使用二分查找寻找 target 的第一个出现位置（左边界）。"""
            left, right = 0, len(nums) - 1
            first_pos = -1
            while left <= right:
                mid = left + (right - left) // 2 # 防止整数溢出
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:  # 当 nums[mid] == target 时
                    first_pos = mid  # 记录下这个可能的位置
                    right = mid - 1  # 继续在左半部分搜索，试图找到更靠前的位置
            return first_pos

        def find_last_position(nums, target):
            """使用二分查找寻找 target 的最后一个出现位置（右边界）。"""
            left, right = 0, len(nums) - 1
            last_pos = -1
            while left <= right:
                mid = left + (right - left) // 2
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:  # 当 nums[mid] == target 时
                    last_pos = mid  # 记录下这个可能的位置
                    left = mid + 1   # 继续在右半部分搜索，试图找到更靠后的位置
            return last_pos

        first = find_first_position(nums, target)

        # 优化：如果连第一个位置都找不到，说明 target 不存在，无需再找最后一个位置。
        if first == -1:
            return [-1, -1]

        last = find_last_position(nums, target)

        return [first, last]

    def search(self, nums: List[int], target: int) -> int:
        """
        在旋转排序数组中搜索一个给定的目标值。
        数组可能在某个未知点上进行了旋转（例如 [0,1,2,4,5,6,7] 可能变为 [4,5,6,7,0,1,2]）。

        Args:
            nums: 旋转后的排序数组。
            target: 要搜索的目标值。

        Returns:
            目标值的索引，如果不存在则返回 -1。
        """
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                return mid

            # 关键在于判断 mid 落在了哪个有序的子数组中
            # 情况 1：左半部分 [left, mid] 是单调递增的
            if nums[left] <= nums[mid]:
                # 检查 target 是否在这个有序的左半部分
                if nums[left] <= target < nums[mid]:
                    right = mid - 1 # target 在左边，收缩右边界
                else:
                    left = mid + 1  # target 不在左边，去右边找
            # 情况 2：右半部分 [mid, right] 是单调递增的
            else:
                # 检查 target 是否在这个有序的右半部分
                if nums[mid] < target <= nums[right]:
                    left = mid + 1  # target 在右边，收缩左边界
                else:
                    right = mid - 1 # target 不在右边，去左边找
        return -1

    def findMin(self, nums: List[int]) -> int:
        """
        在一个可能包含重复元素的旋转排序数组中找到最小值。

        Args:
            nums: 旋转后的排序数组（可能含重复值）。

        Returns:
            数组中的最小值。
        """
        left, right = 0, len(nums) - 1
        
        # 如果数组没有旋转（或只有一个元素），第一个元素就是最小值
        if nums[left] < nums[right]:
            return nums[left]

        # 当 left 和 right 相遇时，循环结束，该位置即为最小值
        while left < right:
            mid = left + (right - left) // 2
            
            # 情况 1: mid 右侧是有序的
            if nums[mid] < nums[right]:
                # 最小值可能就是 mid，或者在 mid 的左侧
                # 所以将 right 收缩到 mid，不能是 mid - 1
                right = mid
            # 情况 2: mid 左侧是有序的（且 mid 本身在左侧递增序列中）
            elif nums[mid] > nums[right]:
                # 最小值一定在 mid 的右侧，因为 mid 所在的部分值更大
                left = mid + 1
            # 情况 3: nums[mid] == nums[right]
            else:
                # 无法判断 mid 在哪一侧，但可以确定 right 不是唯一的最小值
                # （如果是，那么 mid 会小于 right）。因此可以安全地排除 right。
                right -= 1
        
        # 循环结束时，left 指向最小值
        return nums[left]

    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        """
        寻找两个正序数组的中位数。
        此解法通过合并两个数组，然后直接找中位数，时间复杂度为 O(m+n)。
        (注意：存在更优的 O(log(min(m,n))) 的二分查找解法)

        Args:
            nums1: 第一个已排序的数组。
            nums2: 第二个已排序的数组。

        Returns:
            两个数组合并后的中位数。
        """
        m, n = len(nums1), len(nums2)
        
        # 直接合并两个已排序的数组
        merged = sorted(nums1 + nums2)
        total_len = m + n

        # 根据总长度的奇偶性计算中位数
        if total_len % 2 == 0:
            # 偶数个元素：返回中间两个元素的平均值
            mid1 = merged[total_len // 2 - 1]
            mid2 = merged[total_len // 2]
            return (mid1 + mid2) / 2
        else:
            # 奇数个元素：返回中间元素
            return float(merged[total_len // 2])

## 栈

In [ ]:
class Solution:
    def isValid(self, s: str) -> bool:
        """
        检查一个只包含 '(', ')', '{', '}', '[' 和 ']' 的字符串是否有效。
        一个有效的字符串需要满足：
        1. 左括号必须用相同类型的右括号闭合。
        2. 左括号必须以正确的顺序闭合。

        此解法利用了栈（Stack）的“后进先出”（LIFO）特性，非常适合处理括号匹配问题。

        Args:
            s: 输入的括号字符串。

        Returns:
            如果字符串有效，返回 True；否则返回 False。
        """
        # 创建一个哈希表（字典）来存储括号的配对关系。
        # 键是左括号，值是对应的右括号。
        # `(-1, -1)` 是一个“哨兵”或“哑节点”，用于简化代码逻辑，
        # 避免在处理第一个字符是右括号时对空栈进行特殊判断。
        mapping = {"(": ")", "{": "}", "[": "]", -1: -1}

        # 初始化一个栈。预先放入哨兵值-1。
        # 这样，如果遇到的第一个非空字符是右括号，stack.pop()不会导致空栈错误，
        # 而是会弹出-1，其映射值也是-1，与任何右括号都不匹配，从而正确地返回False。
        stack = [-1]

        # 遍历输入字符串中的每一个字符。
        for char in s:
            # 如果当前字符是一个左括号（即它是 mapping 的一个键）。
            if char in mapping:
                # 将该左括号压入栈中。
                stack.append(char)
            # 如果当前字符是右括号（即它不在 mapping 的键中）。
            # `stack.pop()` 会弹出最近压入的左括号。
            # `mapping[...]` 会根据弹出的左括号找到其应匹配的右括号。
            # 然后与当前字符 `char` 进行比较。
            elif char != mapping[stack.pop()]:
                # 如果不匹配，说明括号顺序或类型错误，字符串无效，立即返回 False。
                return False

        # 遍历完整个字符串后，检查栈的状态。
        # 如果栈中只剩下最初放入的哨兵值-1（即长度为1），
        # 说明所有的左括号都已成功匹配并弹出。字符串是有效的。
        # 如果栈的长度大于1，说明有未闭合的左括号，字符串无效。
        return len(stack) == 1

    class MinStack:
        def __init__(self):
            # 计算每个前缀的最小值: nums[List], preMin[List]
            # preMin[i] = min( preMin[i-1], nums[i] )
            self.stack = [(0, inf)]

        def push(self, val: int) -> None:
            self.stack.append((val, min(self.stack[-1][-1], val)))

        def pop(self) -> None:
            self.stack.pop()

        def top(self) -> int:
            return self.stack[-1][0]

        def getMin(self) -> int:
            return self.stack[-1][-1]

    def decodeString(self, s: str) -> str:
        stack = []
        cur_num = 0
        cur_str = ""
        
        for char in s:
            if char.isalpha():
                cur_str += char
            if char.isdigit():
                cur_num = cur_num * 10 + int(char)   # 处理多位数
            elif char == "[":
                # 将当前数字和字符串入栈
                stack.append((cur_str, cur_num))
                cur_str = ""
                cur_num = 0
            elif char == "]":
                # 出栈，拼接结果
                last_str, repeat_num = stack.pop()
                # 此时 cur_str 是下层递归的返回值，将其重复 repeat_num 次，拼接到递归前的 last_str 之后
                cur_str = last_str + cur_str * repeat_num
        
        return cur_str

    def dailyTemperatures(self, temperatures: List[int]) -> List[int]:
        ans = [0] * len(temperatures)
        # 栈内存放索引，维护一共递减的温度序列
        # [-1]表示栈顶元素，此元素的温度大于等于当前温度
        stack = [] # 类似todolist
        for i, t in enumerate(temperatures):
            # 如果当前温度比栈顶那天的温度高
            # 说明找到了栈顶那天的"下一个更高温度"
            while stack and t > temperatures[stack[-1]]:
                j = stack.pop()
                ans[j] = i - j 

            # 将当天的索引入栈，等待未来更高温度来匹配
            stack.append(i)
        return ans

    def largestRectangleArea(self, heights: List[int]) -> int:
        max_area = 0
        # 栈用于存储柱子的索引，初始放入-1方便计算宽度
        stack = [-1]
        # 在 heights 末尾添加一个高度为0的柱子，用于确保最后栈中所有元素都能被处理
        heights.append(0)
        
        # 遍历每一根柱子及其索引
        for right, h in enumerate(heights):
            # 当当前柱子高度小于等于栈顶柱子高度时，说明栈顶柱子可以形成一个矩形的右边界
            while len(stack) > 1 and heights[stack[-1]] >= h:
                # 弹出栈顶索引，计算以该柱子为高度的矩形面积
                i = stack.pop()
                # 左边界为当前栈顶元素的索引
                left = stack[-1]
                # 宽度 = 右边界索引 - 左边界索引 - 1
                width = right - left - 1
                # 更新最大面积
                max_area = max(max_area, heights[i] * width)
            # 当前柱子的索引入栈，作为后续柱子的左边界
            stack.append(right)
        
        # 返回计算得到的最大矩形面积
        return max_area

## 堆

In [ ]:
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = []
        for x in nums:
            heapq.heappush(heap, x)
            if len(heap) > k:
                heapq.heappop(heap)

        return heap[0]

    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        # * 简单实现
        # cnt = Counter(nums)
        # cnt.most_common(k)
        # return [tup[0] for tup in cnt.most_common(k)]

        cnt = Counter(nums)
        heap = []  # min-heap: (freq, num)
        for num, freq in cnt.items():
            if len(heap) < k:
                heapq.heappush(heap, (freq, num))
            elif freq > heap[0][0]:
                heapq.heapreplace(heap, (freq, num))

        return [tup[-1] for tup in heap]


class MedianFinder:

    def __init__(self):
        # 较小的一半 (Max Heap - 存储负值实现)
        self.small = []
        # 较大的一半 (Min Heap)
        self.large = []

    def addNum(self, num: int) -> None:
        # 1. 默认先加入 small 堆（最大值堆）
        # 将负值推入 small 堆，因为它是一个最大堆
        heapq.heappush(self.small, -num)

        # 2. 平衡操作 1：确保 small 堆的堆顶 (最大值) <= large 堆的堆顶 (最小值)
        # 如果 small 堆顶（最大值）大于 large 堆顶（最小值），则需要交换
        if self.small and self.large and (-self.small[0] > self.large[0]):
            # 弹出 small 堆顶
            val = -heapq.heappop(self.small)
            # 推入 large 堆
            heapq.heappush(self.large, val)

        # 3. 平衡操作 2：确保两个堆的大小关系：len(large) 可以比 len(small) 多 1
        # 如果 large 堆比 small 堆多出 2 个元素
        if len(self.large) > len(self.small) + 1:
            # 弹出 large 堆顶 (最小值)
            val = heapq.heappop(self.large)
            # 推入 small 堆（负值）
            heapq.heappush(self.small, -val)
        # 如果 small 堆比 large 堆多出 1 个元素
        if len(self.small) > len(self.large):
            # 弹出 small 堆顶 (最大值)
            val = -heapq.heappop(self.small)
            # 推入 large 堆
            heapq.heappush(self.large, val)

    def findMedian(self) -> float:
        # 奇数个元素：中位数是 large 堆的堆顶
        if len(self.large) > len(self.small):
            return float(self.large[0])
        # 偶数个元素：中位数是 large 堆顶和 small 堆顶的平均值
        elif len(self.large) == len(self.small):
            # small 堆顶是负值，需要取反
            return (self.large[0] + (-self.small[0])) / 2.0
        # 理论上不会出现 small 堆比 large 堆多 2 个元素的情况
        else:
            # 返回 large 堆顶 (实际上会被上面的平衡操作覆盖)
            return float(self.large[0])

## 贪心

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_p = prices[0]
        max_profit = 0

        for p in prices:
            # 买入必须在卖出前，因此先计算当天的利润并更新
            max_profit = max(p - min_p, max_profit)
            # 记录最低价
            min_p = min(p, min_p)
        return max_profit

    def canJump(self, nums: List[int]) -> bool:
        farest_pos = 0

        for i, jump in enumerate(nums):
            # 先判断当前位置是否可到达
            if farest_pos < i:
                return False

            # 更新最远可达位置
            farest_pos = max(farest_pos, i + jump)
        return True

    def jump(self, nums: List[int]) -> int:
        n = len(nums)
        steps = 0   
        # 当前这一步的边界（扫描到这就必须跳）
        farthest = 0
        # 当前这一步的边界（扫描到这就必须跳）
        curr_end = 0

        for i in range(n - 1):
            farthest = max(farthest, i + nums[i])

            # 最远位置已经到达或超过最后一个位置
            if farthest >= n - 1:
                return steps + 1
            # 到达了当前这一步的边界，必须进行下一步跳跃
            if i == curr_end:
                steps += 1
                curr_end = farthest
        return steps
    
    def partitionLabels(self, s: str) -> List[int]:
        # 记录每个字符最后出现的位置
        last_occurrence = {c: i for i, c in enumerate(s)}
        postions = []
        start, end = 0, 0

        for i, c in enumerate(s):
            # 更新当前分区的结束位置
            # 取当前字符最后出现位置和当前分区结束位置的最大值
            end = max(end, last_occurrence[c])
            if i == end:
                postions.append(end - start + 1)
                start = i + 1

        return postions


## 动态规划

In [ ]:
class Solution:
    def climbStairs(self, n: int) -> int:
        if n <= 2:
            return n

        # 状态转移方程：dp[i] = dp[i-1] + dp[i-2]
        # 初始化
        dp = [0] * n
        # 边界条件
        dp[0], dp[1] = 1, 2

        for i in range(3, n):
            dp[i] = dp[i - 1] + dp[i - 2]

        return dp[-1]

    def generate(self, numRows: int) -> List[List[int]]:
        result = []

        for i in range(numRows):
            # 创建当前行，初始填充为1
            row = [1] * (i + 1)

            # 计算当前行中间的值（非边界）
            for j in range(1, i):
                row[j] = result[i - 1][j - 1] + result[i - 1][j]

            result.append(row)
        return result
    
    def rob(self, nums: List[int]) -> int:
        n = len(nums)
        dp = [0] * n
        dp[0] = nums[0]
        dp[1] = max(nums[0], nums[1])

        for i in range(2, n):
            dp[i] = max(dp[i-1], nums[i]+dp[i-2])

        return dp[-1]

1
1,1
1,2,1
1,3,3,1